## **POLICY & GOVERNANCE ANALYSIS**

In [1]:
%run ./_setup.py
import os
import sys

Python: c:\Users\Admin\Desktop\Irrigation Water Requirement Prediction\.venv\Scripts\python.exe
Environment ready
Shape: (10000, 20)
<class 'pandas.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Soil_Type                10000 non-null  str    
 1   Soil_pH                  10000 non-null  float64
 2   Soil_Moisture            10000 non-null  float64
 3   Organic_Carbon           10000 non-null  float64
 4   Electrical_Conductivity  10000 non-null  float64
 5   Temperature_C            10000 non-null  float64
 6   Humidity                 10000 non-null  float64
 7   Rainfall_mm              10000 non-null  float64
 8   Sunlight_Hours           10000 non-null  float64
 9   Wind_Speed_kmh           10000 non-null  float64
 10  Crop_Type                10000 non-null  str    
 11  Crop_Growth_Stage        10000 non-null  str    
 12  Season   

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
df = pd.read_csv("../data/irrigation_prediction.csv")

In [3]:
# Ensure categorical columns are properly typed
categorical_cols = ['Soil_Type', 'Crop_Type', 'Crop_Growth_Stage', 'Season', 
                    'Irrigation_Type', 'Water_Source', 'Mulching_Used', 'Region', 
                    'Irrigation_Need']

for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].astype('category')
    else:
        print(f"Warning: Column '{col}' not found in dataset")

In [4]:
# Policy color palette
policy_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

### *96. Policy Compliance Rates*

In [6]:
print("\n96. Policy Compliance...")

# Create compliance metrics
# Derive Water_Efficiency (used in water_management_plotly_analysis.ipynb and farm_management_plotly_analysis.ipynb)
df['Water_Efficiency'] = df['Soil_Moisture'] / (df['Previous_Irrigation_mm'] + 1)

compliance_data = (
    df.groupby('Region', observed=False)
      .agg(
          Modern_Rate=('Irrigation_Type', lambda s: s.isin(['Drip', 'Sprinkler']).mean() * 100),
          Mulching_Rate=('Mulching_Used', lambda s: s.eq('Yes').mean() * 100),
          Water_Efficiency=('Water_Efficiency', 'mean'),
          Success_Rate=('Irrigation_Need', lambda s: s.eq('Low').mean() * 100)
      )
      .reset_index()
)


fig96 = go.Figure()

# Add gauge-like indicators
for i, metric in enumerate(['Modern_Rate', 'Mulching_Rate', 'Water_Efficiency', 'Success_Rate']):
    fig96.add_trace(go.Indicator(
        mode="gauge+number",
        value=compliance_data[metric].mean(),
        title={'text': metric.replace('_', ' ')},
        domain={'row': i//2, 'column': i%2},
        gauge={'axis': {'range': [0, 100]},
               'bar': {'color': policy_colors[i]}}
    ))

fig96.update_layout(
    grid={'rows': 2, 'columns': 2, 'pattern': "independent"},
    height=500,
    title_text='<b>96. Policy Compliance Dashboard</b><br>Average Performance Indicators'
)
fig96.show()


96. Policy Compliance...


### *97. Subsidy Distribution Equity*

In [7]:
print("\n97. Subsidy Equity...")

# Create subsidy distribution simulation
subsidy_sim = []
for region in df['Region'].unique():
    # Simulate subsidy distribution
    farm_sizes = df[df['Region'] == region]['Field_Area_hectare'].values[:10]
    subsidies = np.random.choice([1000, 2000, 5000], size=len(farm_sizes), 
                                p=[0.6, 0.3, 0.1])  # Bias toward larger farms
    for size, subsidy in zip(farm_sizes, subsidies):
        subsidy_sim.append({'Region': region, 'Farm_Size': size, 'Subsidy': subsidy})

subsidy_df = pd.DataFrame(subsidy_sim)

# Calculate Gini coefficient proxy
def gini_coefficient(values):
    sorted_vals = np.sort(values)
    n = len(sorted_vals)
    index = np.arange(1, n + 1)
    return (np.sum((2 * index - n - 1) * sorted_vals)) / (n * np.sum(sorted_vals))

equity_stats = subsidy_df.groupby('Region', observed=False).apply(
    lambda x: pd.Series({
        'Avg_Subsidy': x['Subsidy'].mean(),
        'Inequality_Index': gini_coefficient(x['Subsidy']) * 100
    })
).reset_index()

fig97 = px.bar(equity_stats, x='Region', y='Inequality_Index',
               color='Avg_Subsidy', color_continuous_scale='RdYlGn_r',
               title='<b>97. Subsidy Distribution Inequality</b><br>Lower is More Equitable',
               labels={'Inequality_Index': 'Inequality Index (%)'})

fig97.update_layout(height=500, plot_bgcolor='white')
fig97.show()


97. Subsidy Equity...


### *98. Conflict Resolution Mechanisms*

In [9]:
print("\n98. Water Conflict Resolution...")

# Simulate conflict resolution data
conflict_data = []
mechanisms = ['Direct Negotiation', 'Village Council', 'Government Mediation', 'Court']
for region in df['Region'].unique():
    for mechanism in mechanisms:
        # Simulate success rates
        success_rate = np.random.uniform(60, 95)
        resolution_time = np.random.uniform(1, 12)  # months
        conflict_data.append({
            'Region': region,
            'Mechanism': mechanism,
            'Success_Rate': success_rate,
            'Resolution_Time': resolution_time
        })

conflict_df = pd.DataFrame(conflict_data)

fig98 = px.scatter(conflict_df, x='Resolution_Time', y='Success_Rate',
                   color='Mechanism', size='Success_Rate',
                   facet_col='Region', facet_col_wrap=2,
                   title='<b>98. Water Conflict Resolution Effectiveness</b><br>Top-right = Fast & Effective',
                   labels={'Resolution_Time': 'Time to Resolve (months)',
                          'Success_Rate': 'Success Rate (%)'})

fig98.update_layout(height=600, plot_bgcolor='black')
fig98.show()


98. Water Conflict Resolution...


### *99. Stakeholder Influence Mapping*

In [10]:
print("\n99. Stakeholder Influence...")

# Define stakeholders and their influence levels
stakeholders = ['Farmers', 'Water Dept', 'Local Gov', 'NGOs', 'Industry', 'Research']
influence_data = []

for region in df['Region'].unique():
    for stakeholder in stakeholders:
        # Base influence with regional variations
        base_influence = {
            'Farmers': 70, 'Water Dept': 85, 'Local Gov': 75,
            'NGOs': 60, 'Industry': 65, 'Research': 55
        }
        # Add regional variation
        influence = base_influence[stakeholder] + np.random.uniform(-15, 15)
        influence_data.append({
            'Region': region,
            'Stakeholder': stakeholder,
            'Influence_Score': min(100, max(0, influence))
        })

influence_df = pd.DataFrame(influence_data)

fig99 = px.bar(influence_df, x='Stakeholder', y='Influence_Score',
               color='Region', barmode='group',
               title='<b>99. Stakeholder Influence by Region</b>',
               labels={'Influence_Score': 'Influence Score (0-100)'})

fig99.update_layout(height=500, plot_bgcolor='white')
fig99.show()


99. Stakeholder Influence...


### *100. Policy Impact Assessment*

In [ ]:
print("\n100. Policy Impact Assessment...")

# Simulate policy interventions
policies = ['Water Pricing', 'Subsidy Reform', 'Tech Adoption', 'Training', 'Infrastructure']
policy_impacts = []

for region in df['Region'].unique():
    region_df = df[df['Region'] == region]
    base_success = (region_df['Irrigation_Need'] == 'Low').mean() * 100
    
    for policy in policies:
        # Simulate policy impact
        impact_factor = {
            'Water Pricing': 1.15,
            'Subsidy Reform': 1.20,
            'Tech Adoption': 1.25,
            'Training': 1.10,
            'Infrastructure': 1.30
        }
        
        # Add random variation
        impact = base_success * impact_factor[policy] + np.random.uniform(-5, 10)
        
        policy_impacts.append({
            'Region': region,
            'Policy': policy,
            'Pre_Policy': base_success,
            'Post_Policy': min(100, impact),
            'Impact': min(100, impact) - base_success
        })

policy_df = pd.DataFrame(policy_impacts)

fig100 = px.bar(policy_df, x='Policy', y='Impact', color='Region',
                barmode='group', title='<b>100. Policy Impact Assessment</b>',
                labels={'Impact': 'Success Rate Improvement (%)'})

fig100.update_layout(height=500, plot_bgcolor='white')
fig100.show()


100. Policy Impact Assessment...
